In [69]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from googletrans import Translator
import unicodedata
import pycountry
import pycountry_convert as pc
from difflib import get_close_matches



In [ ]:
pip install pycountry-convert

In [3]:
def fetch_and_parse_page(url):
    """
    Récupère et parse le contenu HTML d'une page web.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        # Récupérer la page
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lève une exception pour les erreurs HTTP
        
        # Parser le contenu avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de l'URL {url}: {e}")
        return None

def save_html_to_txt(soup, filename,dir_path='C:/Users/glenn/OneDrive/Bureau/VScode saves/WebScrapping/Projet'):
    """
    Sauvegarde le contenu HTML formaté dans un fichier texte.
    """
    path=f'{dir_path}/{filename}.txt'
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(soup.prettify())
        print(f"HTML sauvegardé dans le fichier : {filename}")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde du fichier : {e}")

def extract_text_by_class(soup, balise,class_name):
    """
    Récupère tous les textes des balises <span> ayant une classe spécifique.
    """
    # Chercher toutes les balises <span> avec la classe donnée
    spans = soup.find_all(balise, class_=class_name)
    
    # Extraire et retourner le texte
    return [span.get_text(strip=True) for span in spans]

Lonely planet extract

In [4]:
def LonelyPlanet_attractions(soup):
    
    texts = extract_text_by_class(soup,"span", "heading-05 font-semibold")
    df_LonelyPlanet=pd.DataFrame({'Title': texts})
    df_LonelyPlanet.insert(0, 'site', 'LonelyPlanet')
    df_LonelyPlanet['rank'] = range(len(df_LonelyPlanet))
    return df_LonelyPlanet


Bucket List extract

In [5]:
def BucketList_attractions(soup):

    df_BucketList=pd.DataFrame()
    # Trouver toutes les balises <article>
    articles = soup.find_all('article', class_='listing-card bg-white shadow-listing')

    # Initialiser une liste pour stocker les résultats

    for article in articles:
        # Extraire le titre de la balise <h2> (nom de l'attraction)
        title_tag = article.find('h2', class_='text-2xl md:text-3xl font-bold')
        title = title_tag.get_text(strip=True) if title_tag else 'Titre non trouvé'

        # Initialiser un dictionnaire pour stocker les informations de l'attraction
        attraction_info = {'Title': title}

        # Trouver toutes les balises <p> avec les informations sur la durée, l'âge, etc.
        p_tags = article.find_all('p', class_='flex items-center space-x-1 text-lg')

        for p in p_tags:
            # Extraire le nom de la catégorie (par exemple "Duration", "Good for age", etc.)
            label_tag = p.find_all('span')[1]
            if label_tag:
                label_value = label_tag.get_text(strip=True).split(':')
                if len(label_value)==2:
                    label=label_value[0]
                    value=label_value[1]
                    attraction_info[label] = value

        # Ajouter l'attraction à la liste des résultats
        df_BucketList = pd.concat([df_BucketList, pd.DataFrame([attraction_info])], ignore_index=True)
    df_BucketList.insert(0, 'site', 'BucketList')
    df_BucketList['rank'] = range(len(df_BucketList))
    return df_BucketList



WorldTravelGuide extract

In [6]:
def WorldTravelGuide_attractions(soup):

    df_WorldTravelGuide=pd.DataFrame()
    articles = soup.find_all('div', class_='high')
    articles.extend(soup.find_all('div', class_='medium'))

    for article in articles:
        # Extraire le titre de la balise <h2> (nom de l'attraction)
        title_tag = article.find('h3')
        title = title_tag.get_text(strip=True) if title_tag else 'Titre non trouvé'

        # Initialisation du dictionnaire pour stocker les informations extraites
        attraction_info = {'Title': title}

        # Extraire la description
        description_tag = article.find('p')
        if description_tag:
            attraction_info['description'] = description_tag.get_text(strip=True)

        # Extraire l'adresse
        address_tag = article.find('b', string="Address: ")
        if address_tag:
            address = address_tag.find_next('span')
            if address:
                attraction_info['Address'] = address.get_text(strip=True)

        # Extraire les horaires d'ouverture
        opening_times_tag = article.find('b', string="Opening times: ")
        if opening_times_tag:
            opening_times = opening_times_tag.find_next('p')
            if opening_times:
                attraction_info['Opening times'] = opening_times.get_text(strip=True)

        # Extraire le site Web
        website_tag = article.find('b', string="Website: ")
        if website_tag:
            website = website_tag.find_next('a')
            if website and website.get('href'):
                attraction_info['Website'] = website.get('href')

        # Extraire les frais d'admission
        admission_fees_tag = article.find('b', string="Admission Fees: ")
        if admission_fees_tag:
            admission_fees = admission_fees_tag.find_next('p')
            if admission_fees:
                attraction_info['Admission Fees'] = admission_fees.get_text(strip=True)

        # Extraire l'accès handicapé
        disabled_access_tag = article.find('b', string="Disabled Access: ")
        if disabled_access_tag:
            #comment récupérer le texte juste après disabled_access_tag
            disabled_access_text = disabled_access_tag.next_sibling.strip() if disabled_access_tag.next_sibling else 'Non spécifié'
            attraction_info['Disabled Access'] = disabled_access_text
        df_WorldTravelGuide = pd.concat([df_WorldTravelGuide, pd.DataFrame([attraction_info])], ignore_index=True)

    df_WorldTravelGuide.insert(0, 'site', 'WorldTravelGuide')
    df_WorldTravelGuide['rank'] = range(len(df_WorldTravelGuide))
    return df_WorldTravelGuide

In [7]:
def CNTraveler_attractions(soup):
    df_CNTraveler=pd.DataFrame()
    articles = soup.find_all('div', class_='GallerySlideFigCaption-dOeyTg gWbVWR')
    for article in articles:
        # Extraire le titre de l'attraction
        title_tag = article.find('span',class_='GallerySlideCaptionHedText-iqjOmM jwPuvZ')
        title = title_tag.get_text(strip=True) if title_tag else 'Titre non trouvé'
        attraction_info = {'Title': title}
        description_tag = article.find('p')
        if description_tag:
            attraction_info['description'] = description_tag.get_text(strip=True)
        df_CNTraveler = pd.concat([df_CNTraveler, pd.DataFrame([attraction_info])], ignore_index=True)
    df_CNTraveler.insert(0, 'site', 'CNTraveler')
    df_CNTraveler['rank'] = range(len(df_CNTraveler))
    return df_CNTraveler

In [8]:
def Routard_attractions(soup):
    df_Routard=pd.DataFrame()
    articles = soup.find_all('div', class_='bg-rtd-grey-100 flex h-96 w-60 flex-col rounded-xl p-4')
    for article in articles:
        # Extraire le titre de l'attraction
        title_tag = article.find('h2',class_='group-hover:text-rtd-green my-2 font-semibold')
        title = title_tag.get_text(strip=True) if title_tag else 'Titre non trouvé'
        attraction_info = {'Title': title}
        description_tag = article.find('div', class_='rtd-wysiwyg line-clamp-3')
        if description_tag:
            attraction_info['description'] = description_tag.get_text(strip=True)
        df_Routard = pd.concat([df_Routard, pd.DataFrame([attraction_info])], ignore_index=True)
    df_Routard.insert(0, 'site', 'Routard')
    df_Routard['rank'] = range(len(df_Routard))
    return df_Routard

In [81]:
def translate_location(name, src_lang="en", dest_lang="fr"):

    translator = Translator()
    try:
        translation = translator.translate(name, src=src_lang, dest=dest_lang).text
        print(translation)
        translation= translation.replace("'","-").replace(" ","-").lower()
        #suppression des accents
        translation = unicodedata.normalize('NFD', translation)
        text = ''.join(char for char in translation if unicodedata.category(char) != 'Mn')
    
        return text
    except Exception as e:
        print(f"Error during translation: {e}")
        return name

In [86]:



def country_to_continent(country_name):
    country_names = [country.name for country in pycountry.countries]
    country = get_close_matches(country_name, country_names,n=1)
    if len(country) == 1:
        country_alpha2 = pc.country_name_to_country_alpha2(country[0])
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name.lower()
    else: return None


In [84]:
def routard_city_to_region(city):
    cities_to_regions = {
        "strasbourg": "alsace",
        "bordeaux": "aquitaine-bordelais-landes",
        "rennes": "bretagne",
        "nice": "cote-d-azur",
        "paris": "ile-de-france",
        "montpellier": "languedoc-roussillon",
        "toulouse": "midi-toulousain-occitanie",
        "lille": "nord-pas-de-calais",
        "nantes": "pays-de-la-loire",
        "marseille": "provence"
    }
    return cities_to_regions[city]

def routard_continent(continent):
    routard_continent_fr = {
        "europe":"europe",
        "africa":"afrique",
        "north america":"ameriques",
        "south america":"ameriques",
        "asia":"asie",
        "oceania":"oceanie"
    }
    return routard_continent_fr[continent]


In [ ]:
def main(country='france',city='paris',websites_to_call=['Routard','WorldTravelGuide','BucketList','LonelyPlanet','CNTraveler']):
    city= str(city).lower().replace("'","-").replace(" ","-")

    continent = country_to_continent(country)
    continent_fr = routard_continent(continent)
    country_fr = translate_location(country)
    city_fr = translate_location(city)

    URL_dict={
        'LonelyPlanet':f'https://www.lonelyplanet.com/{country}/{city}/attractions',
        'BucketList':f'https://www.bucketlisttravels.com/destination/{city}/best-things-to-see-and-do',
        'WorldTravelGuide':f'https://www.worldtravelguide.net/guides/{continent}/{country}/{city}/things-to-see/',
        'CNTraveler':f'https://www.cntraveler.com/gallery/best-things-to-do-in-{city}',
        'Routard':f'https://www.routard.com/fr/guide/top/{continent_fr}/{country_fr}/{city_fr}'
    }
    if country=='france':
        region = routard_city_to_region (city_fr)

        URL_dict['Routard']=f'https://www.routard.com/fr/guide/top/{country}/{region}/{city_fr}'

    URL_extractor={
        'LonelyPlanet_attractions': LonelyPlanet_attractions,
        'BucketList_attractions':BucketList_attractions,
        'WorldTravelGuide_attractions':WorldTravelGuide_attractions,
        'CNTraveler_attractions':CNTraveler_attractions,
        'Routard_attractions':Routard_attractions
    }
    df=pd.DataFrame()
    for website in websites_to_call:
        soup=fetch_and_parse_page(URL_dict[website])
        #save_html_to_txt(soup,f'{website}_{city}')
        if soup:
            df=pd.concat([df,URL_extractor[f'{website}_attractions'](soup)],ignore_index=True)
    return (df)
        

df_main=main("germany", "berlin")

In [ ]:
df_main

## NLP regroupement

In [94]:
from googletrans import Translator

# Initialiser le traducteur
translator = Translator()

def detect_language(text):
    try:
        detected = translator.detect(text)
        return detected.lang  # Retourne le code langue (ex: 'fr')
    except Exception as e:
        return f"Erreur lors de la détection : {e}"

def translate_to_english(text):
    status=False
    lang = detect_language(text)
    if lang == 'en':
        return (text,status)  # Aucun besoin de traduction
    try:
        translated = translator.translate(text, dest='en')
        status=True
        return (translated.text,status)
    except Exception as e:
        return f"Erreur lors de la traduction : {e}"

# Appliquer la fonction de traduction à la colonne 'Title' et créer une nouvelle colonne 'Title_english'
for index in df_main.index:
    translation=translate_to_english(df_main.at[index, 'Title'])
    df_main.at[index, 'Title_english'] = translation[0]
    df_main.at[index, 'VO'] = translation[1]

In [ ]:
import nltk 
from unidecode import unidecode
from nltk.corpus import stopwords
import re

country='france'
city='paris'
# Télécharger les stopwords français
nltk.download('stopwords')
stop_words = set(stopwords.words('french') + stopwords.words('english'))
stop_words.update([country, city])

def remove_stopwords_and_accents(text):
    # Supprimer les accents
    text = unidecode(text)
    # Supprimer les caractères spéciaux
    text = re.sub(r'\W+', ' ', text).lower()
    # Supprimer les stopwords

    # Ensemble pour suivre les mots déjà rencontrés
    unique_words = []  # Liste pour stocker les mots uniques
    # Parcourir chaque mot dans le texte
    for word in text.split():
        # Convertir le mot en minuscules pour ignorer la casse
        # Si le mot n'est pas un stop word et qu'il n'a pas encore été rencontré
        if word not in stop_words and word not in unique_words:
            unique_words.append(word)
    # Recomposer la chaîne de mots uniques
    return ' '.join(unique_words)

# Appliquer la fonction à la colonne 'Title' et créer une nouvelle colonne 'Title_cleaned'
df_main['Title_english'] = df_main['Title_english'].apply(remove_stopwords_and_accents)

In [96]:
from rapidfuzz import fuzz

# Fonction personnalisée pour mixer token_sort_ratio et partial_token_sort_ratio
def combined_token_sort_ratio(s1, s2, **kwargs):
    # Calculer les scores individuels
    score_token_sort = fuzz.token_sort_ratio(s1, s2)
    score_partial_token_sort = fuzz.partial_token_sort_ratio(s1, s2)
    
    # Mélanger les scores (par exemple, moyenne pondérée)
    return 0.4 * score_token_sort + 0.6 * score_partial_token_sort

def extractOne(query, grouped_df, scorer=fuzz.ratio, processor=None, score_cutoff=None):
    """
    Recherche l'élément le plus similaire à une chaîne donnée dans une liste de choix.

    Parameters:
    query (str): La chaîne de recherche.
    choices (iterable): Les choix possibles pour la correspondance.
    scorer (callable): Fonction de similarité, par défaut `fuzz.ratio`.
    processor (callable): Fonction de prétraitement pour les chaînes (par exemple, str.lower).
    score_cutoff (float): Score minimum pour accepter une correspondance.

    Returns:
    tuple: Le choix le plus proche et son score, ou None si aucun choix n'est au-dessus de `score_cutoff`.
    """
    if processor:
        query = processor(query)

    best_match = None
    best_score = score_cutoff if score_cutoff is not None else 0

    for index in grouped_df.index:
        choice = grouped_df.loc[index,'Title_english']
        # Applique le prétraitement si un processor est défini
        processed_choice = processor(choice) if processor else choice
        # Calcule le score en utilisant le scorer
        score = scorer(query, processed_choice)
        # Garde la meilleure correspondance
        if score > best_score:
            best_match = index
            best_score = score

    # Retourne le meilleur résultat, ou None si aucun ne dépasse score_cutoff
    return (best_match, best_score) if best_match is not None else None

# Fonction pour trouver les correspondances similaires
def find_similar_groups(df, threshold=80):
    df['group']=None
    last_group=0
    for index in df.index:
        #crée un sous df avec les Title_english qui ont une valeur différente de None dans la colonne group et une valeur de site différente de celle de l'index
        sub_df=df[(df['group'].notnull()) & (df['site']!=df.loc[index,'site'])]
        # Utilisation de la fonction personnalisée comme scorer
        match = extractOne(df.loc[index,'Title_english'], sub_df, scorer=combined_token_sort_ratio)
        if match and match[1] >= threshold:
            df.loc[index,'group']=df.loc[match[0],'group']
        else:
            df.loc[index,'group']=last_group
            last_group+=1
    return df

df_main = find_similar_groups(df_main, threshold=80).sort_values(by='group', ascending=True, inplace=False)
#fonctionne bien sauf pour 'triomph arch' -> à corriger

In [ ]:
df_main

In [ ]:
def merge_groups(df):
    merged_df = pd.DataFrame()
    
    for group in df['group'].unique():
        group_df = df[df['group'] == group]
        
        # Garder le titre le plus court qui a VO == True
        filtered_group_df = group_df[group_df['VO'] == True]
        if not filtered_group_df.empty and filtered_group_df['Title'].notnull().any():
            shortest_title_row = filtered_group_df.loc[filtered_group_df['Title'].str.len().idxmin()]
        else:
            shortest_title_row = group_df.loc[group_df['Title'].str.len().idxmin()]  # ou un comportement par défaut
        
        # Fusionner les différentes lignes
        merged_row = shortest_title_row.copy()
        merged_row['count'] = len(group_df)
        merged_row['rank'] = group_df['rank'].mean()
        
        # Ajouter la ligne fusionnée au DataFrame final
        merged_df = pd.concat([merged_df, pd.DataFrame([merged_row])], ignore_index=True)
    
    return merged_df

df_merged = merge_groups(df_main).sort_values(by=['count', 'rank'], ascending=[False, True], inplace=False)
df_merged.reset_index(drop=True, inplace=True)
df_merged